# Artificial Intelligence Lab
Artificial intelligence is applied data analysis.

## Objectives
In this module, you'll explore ...

- The role of Large Language Models (LLMs) in data analysis
- The risks and ethics of AI
- Popular AI integration methods
- Free and open-source AI tools

## Introduction
From a very broad perspective, there are two kinds of artificial intelligence:

**Generative** AI, which generates new samples from existing data. For example, ChatGPT uses its training data to generate responses that resemble human conversation.

**Discriminative** AI, which classifies provided samples into categories based on existing data. For example, many image classification models can be used to identify the subject matter in a picture.

Most of today's popular AIs are generative in nature, and today's lesson is about them.

## History
The term "AI", or "artificial intelligence", has a long history with many steps in its evolution. Before the rise of commercial large language models such as Google's Gemini, OpenAI's GPT, or Meta's LLaMa, "AI" was still a somewhat-common phrase. It could refer to:

- Machine learning and neural networks, two techniques for allowing a computer algorithm to have more accurate results through repeated experimentation.

- Assistive analytical tools, such as algorithms for helping doctors identify tumors in cancer screenings, or large companies fine-tuning their logistics with predictive analysis.

- Decision trees for computer-controlled video game characters, or complex algorithms like the [A* search algorithm](https://en.wikipedia.org/wiki/A*_search_algorithm), used for pathfinding.

## Large Language Models
**Large language models**, or LLMs, have become popularized as the most widely-available form of artificial intelligence. They require large amounts of natural language data, called a "**corpus**", to simulate intelligence by tracking the relationships between words.

# Activity 1: Small Language Model
Can a computer write Shakespeare?

Let's find out!

We are going to experiment with language analysis using a concept called **Markov chains**.

It might sound intimidating, but we can actually create this model using basic Python tools we're already familiar with such as lists, dictionaries, and loops. No `pip install` required!

<center>

![A Markov chain](txt-markov.svg)

</center>

Markov chains are simple, but close enough to large-scale language models to help us form an idea of how those LLMs work.

In our code, we will read two words at a time. Each pair of words is like a row in our model's database. When we use a couple of words to "prompt" the model, it will use those pairs to determine how to respond, by "predicting" which word should come next based on all the word pairs it has already encountered.

We'll begin with a **plain text corpus** of Shakespeare's writing (~5MB) adapted from data freely provided by [OpenSourceShakespeare](https://www.opensourceshakespeare.org/). Run the cell below to read the contents of the training corpus into the `training_text` variable.

In [1]:
# Run this cell to read the training corpus from your computer's filesystem
# into this Jupyter Notebook's working memory.

file_handle = open("ShakespeareCorpus.txt", "r") # Open the file in read mode ("r").
training_text = file_handle.read() # Read the contents into one big giant string
file_handle.close() # Close the file.
print(f"Finished loading {len(training_text.split()):,} words of training data into memory.")

Finished loading 883,531 words of training data into memory.


Our language model will represent the possible words that come after each word in the training corpus.

Let's **train our model**, which we will save into a Python dictionary in the variable `possibles`.

(This cell might take a few minutes to run!)

In [2]:
# Run this cell to train your model (only need to do it once)

# Based on Ben Hoyt's sample code:
# https://benhoyt.com/writings/markov-chain/

# Initialize the two variables we'll use to walk the training text
word1 = ''
word2 = ''

# Our model is a dictionary indexed by pairs of words.
possibles = {}
all_training_lines = training_text.split("\n")

for each_line in training_text.split("\n"):
    for each_word in each_line.split(" "):
        if each_word == "":
            continue

        # Initialize new pairs with an empty list.
        if (word1, word2) not in possibles:
            possibles[word1, word2] = []

        possibles[word1, word2].append(each_word)

        word1, word2 = word2, each_word

print(f"Model training complete.")

import sys
size_of_model_in_bytes = sys.getsizeof(possibles)
size_of_model_mb = size_of_model_in_bytes / 1000 / 1000
print(f"The model takes up {round(size_of_model_mb, 2)} megabytes (MB) in memory.")

Model training complete.
The model takes up 20.97 megabytes (MB) in memory.


Now we will prime, or **prompt** it, with some words so it knows where to start.

We'll also set some other configuration variables, like the minimum and maximum length of the answer and what constitutes "punctuation" in terms of ending the output.

And then we will define our generative algorithm, which can be called any number of times to generate new results.

In [40]:
# Run this cell to set your prompt or change configuration variables.

minimum_answer_length_in_words = 8
maximum_answer_length_in_words = 15
punctuation = ['.', '?', '!']
prompt = input("Choose a two-word prompt, or press Enter for random: ")

import random

if prompt:
    prompt1, prompt2 = prompt.split(" ")
    print(f"Chosen prompt:", end="")
else:
    uppercase_pairs = [word for word in possibles if word[0][:1].isupper()]
    prompt1, prompt2 = random.choice(uppercase_pairs)
    print(f"Random prompt:", end="")

print(f" {prompt1} {prompt2}")
print(f"Output length: Between {minimum_answer_length_in_words} and {maximum_answer_length_in_words} words")

# Define our generative algorithm
def generate_output(base_word, next_word):
    output = [base_word, next_word]
    index = 0

    while True:
        try:
            word = random.choice(possibles[base_word, next_word])
        except KeyError:
            output = f"\"{base_word} {next_word}\"? What is this sorcery?".split()
            break

        if word == output[-1]:
            break # Bail out if we tried again and got the same word

        output.append(word)

        ending_punctuation = word[-1] in punctuation
        met_minimum = index >= minimum_answer_length_in_words
        exceeded_maximum = index > maximum_answer_length_in_words

        if met_minimum:
            if ending_punctuation:
                break # We're done!
            elif exceeded_maximum:
                continue # Try again

        base_word, next_word = next_word, word
        index += 1

    return output

Chosen prompt: Measure for
Output length: Between 8 and 15 words


Now, finally, we generate our output:

In [44]:
# Run this cell to write some Shakespeare!

import textwrap
output = generate_output(prompt1, prompt2)
print(textwrap.fill(' '.join(output)))

Measure for measure must be conclusions. Well, I cannot blame thee for
ever.


Results

The king is my maid's garments: he upon whose deck The sea-tost
Pericles appears to me, For I have spake did have repent have am will
do must have fear am had am can come have can have cannot have can am
do will have do mine must have

Thou art arm'd, Gloucester. Let the foils be brought, From limits far
remote where thou Wast shot at with them, with

Thou art proud, Apemantus. Of nothing so much shall I find thou art
more deep damn'd than Prince Lucifer:

Thou art sworn, Eros, That, when we rage, advice is often thus, And
prays that you three will but

"Dear Othello"? What is this sorcery?

Measure for measure must be her mild companion. You gods that hear me,
This sword hath ended in the

- Try generating with the same prompt several times.
- Try experimenting with a few different prompts. Here are some ideas:
    - The king (lowercase k)
    - The King (capital K)
    - O gull!
    - Hark, wretches!
    - She thinks
    - But kindness,
- Try specifying a prompt with words that aren't in the training data. What happens?
- Try changing the minimum and maximum settings and re-prompting.

Are there prompts that almost always have similar results?

What about prompts that are very different every time?

Why would some prompts be more random than others?

# Risks and Ethics
The Shakespeare corpus is about 5 megabytes, and contains around 900,000 words.

With enough good training data, even our rudimentary language model consisting of less than 100 lines of Python is able to produce noteworthy results that can occasionally even pass for the real thing.

Imagine how flexible our model would be if we used 5,000,000 megabytes of data instead, containing billions or trillions of words, and spent time refining the generation code to clean up edge cases like mid-sentence punctuation and stage directions.

Perhaps you can start to imagine why large language models work the way they do. They are trained, in some cases, on the *entire Internet*--so they can respond to any prompt that has ever been typed anywhere online.

But, doesn't all that data belong to someone else?

And what about how much power it takes to run an AI datacenter?

Like all new technologies, large language models are controversial. No one can predict exactly how a new technology will change the world, but LLMs have already shaped ours in some ways that will have far-reaching consequences.

## Separating Hype from Reality
**The real magic is in the data**.

Language models are useful when they're trained on good, high-quality data, like recent textbooks, professional artwork, or highly-upvoted answers on technical forums.

Your idea of how LLMs work will inform your ability to know how to prompt effectively and interpret AI output reliably.

It's critical to keep in mind that, while modern generative AIs may occasionally provide impressive results, **they are *not* actually capable of reasoning, learning, arguing, designing, or anything other than reproducing different combinations of their input data**.

## Controversy
All technology is political in nature. Things are developed in a certain way for many reasons. As a data analysis tool, large language models are not inherently harmful, but they can be deployed maliciously, and their usage can have harmful results even if there is no intention to do so.

Almost everyone agrees that AIs have their uses. However, some aspects make AI a tough sell when viewed from a long-term perspective:

A ChatGPT query uses [up to 25 times more energy than a Google search](https://www.brusselstimes.com/world-all-news/1042696/chatgpt-consumes-25-times-more-energy-than-google), and a single conversation may use as much as a half-liter of water in server cooling, and the emissions of their data centers have also been [cause for concern](https://www.theguardian.com/technology/2024/sep/15/data-center-gas-emissions-tech).

LLMs require staggering amounts of data to function. This is resulting in a great shift in the availability of data on the Internet. Some previously open source projects have already [shut down](https://github.com/rspeer/wordfreq/blob/master/SUNSET.md) in protest of AI providers' handling of how they scrape their data off the Internet.

This also means that you must be careful with the data you share to an AI. You cannot guarantee, even if [OpenAI](https://help.openai.com/en/articles/7842364-how-chatgpt-and-our-language-models-are-developed#h_2df02d4917) or [Anthropic](https://www.anthropic.com/uk-government-internal-ai-safety-policy-response/data-input-controls-and-audit) states it is so, that that data is secure, and if it even accidentally gets passed into training data uncleaned, the resulting data leak can mean an entire company folds, or someone's life is ruined.

Additionally, it is not really known exactly what is in any given training corpus, and [many concerns](https://spectrum.ieee.org/midjourney-copyright) have been [raised](https://knowingmachines.org/models-all-the-way) about not just the quality, but the legality and morality, of certain data used to train models.

## Foundational Knowledge Still Rules
AI offers efficiency and convenience, but it's not a shortcut to success. Foundational knowledge of programming (or any topic you are using AI to help with) remains *essential* for you to use AI tools responsibly and enhance your learning journey.

**AI should be used as a learning aid, not a "cheat code".**

## Don't Rely Solely on AI
AI lacks the ability to fully understand your unique needs and context. Relying solely on AI for learning can hinder the development of essential cognitive skills. By leaning too much on AI, you may become a passive learner, lacking the ability to think critically and independently when you depend only on AI for problem-solving; this is problematic, as that is in fact one of AI's weaknesses, not strengths!

[A 2024 article on automation bias - Brainfacts.org](https://www.brainfacts.org/neuroscience-in-society/tech-and-the-brain/2024/why-ai-can-push-you-to-make-the-wrong-decision-at-work-090324)

AI systems are built on vast datasets, and if these datasets contain biases, it can perpetuate unfairness and discrimination. You must be mindful of potential biases present in AI algorithms. AI can provide solutions based on existing patterns, but lacks the capacity for imagination and thinking "outside the box." You must foster personal habits that promote human creativity *alongside* AI support!

# Activity 2: How to learn with LLMs
## WARNING!
**Do not** put anything into the AI prompt that you wouldn't want to see posted on social media.

Prompts are used to continue training the model, so if you ask ChatGPT to proofread your novel (for example), the entire text of your book will potentially become part of ChatGPT's training data, meaning it could be reproduced in whole or in part with no attribution to you.

https://chatgpt.com

## Prompt Ideas
1. "Explain this concept like I'm 5" - drop a concept you want to know more on, then vet the response against the Python documentation. Was it right?
2. "Add comments to this code to explain what each line does"; try pasting code from a curriculum file, or try pasting markov.py from above!
3. "Write a program to clean this data"; put some data from your dataset and see if anything interesting happens.
4. "Review this code for possible improvements"; an easy one!
5. "Write a project plan for the following program"; writing outlines, plans, and action steps is a place where AI shines because most processes have been done before and someone somewhere has a tutorial on it!
6. Best practices for prompting:
    - Be concise, but detailed. Avoid flourishy words or adjectives and focus on nouns and verbs... data and actions to be performed on the data.
    - Break things into smaller concepts and processes - you can use the model to help you here. Ask it to break the concept into smaller subconcepts - you can do this recursively as many times as you need to get to granular enough a level that it gives you accurate output
    - The model knows literally nothing. The more detail you give it surrounding the specific context - the RELEVANT specific context, not external "noise" that may distract it from the focus of your query - the more likely its output will be appropriate to your context.
    - To that end, if you are working with a model which allows adding context or passing in files, this is **extremely** useful in dramatically increasing the accuracy and decreasing hallucinations or irrelevant results. **JUST REMEMBER THAT YOU CANNOT ASSUME THAT DATA WILL STAY PRIVATE.**
7. AI examples:
    - [Perplexity](https://www.perplexity.ai/) is great for up to date web research, especially if you need cited sources.
    - The ubiquitous [ChatGPT](https://chatgpt.com/) is fast and efficient and a great "swiss army knife"
    - [Claude Opus](https://claude.ai/) is comparable to ChatGPT in efficiency and accuracy but more emotionally intelligent in it's responses and often a more eloquent writer. It's useful for rough drafts of content, and working as a content editor or design assistant.
    - Bing chat (Now [Windows CoPilot](https://www.microsoft.com/en-us/microsoft-copilot/)) is essentially a trained, web-accessing wrapper for ChatGPT, but integrates with Windows 11 system tools to pull off some neat automations if you care to set them up.
    - [Gemini](https://gemini.google.com/app), while more anecdotally more prone to hallucination or misunderstanding of context, is integrated with much of Google's ecosystem, allowing for integration features that feel somewhat intuitive, such as calendar events from emails, emails from docs, docs from events, etc. Web Search summaries have been controversial and therefore scaled back, but Google continues to innovate the integration and automation aspects of it's competitive edge
    - [Phind](https://www.phind.com/) is a promising site uniquely focused on code - the pro here is that it's likely to give you much more detailed, much more accurate responses. The con here is that when it doesn't know the answer or doesn't have enough context, what it gives you is likely to be VERY CONVINCING, and possibly even fully functioning, incorrect code, and it will be absolutely sure it's correct, because the code DOES WORK... just not for what YOU needed.

# Popular AI Platforms
## Free and Open Source AI Tools
Several free AI tools provide powerful capabilities for integrating AI into data analysis projects. Some popular options include:

- **[Tensorflow](https://www.tensorflow.org/)**: An open-source platform for machine learning developed by Google. It offers a comprehensive ecosystem of tools, libraries, and community resources.
- **[Keras](https://keras.io/)**: An open-source Python library aimed at the development and evaluation of neural networks within machine learning and deep learning models. It is capable of running on top of Theano and Tensorflow, which means it can train neural networks with little code. 
- **[PyTorch](https://pytorch.org/)**: A data science library that can be integrated with other Python libraries, such as NumPy. The library can create computational graphs that can be changed while the program is running. It is especially useful for ML and DL applications like natural language processing (NLP) and computer vision.

## [Ollama](https://www.ollama.com/)
Allows for running models fully locally--sometimes, without even a GPU!

## [Jupyter AI](https://jupyter-ai.readthedocs.io)
Jupyter AI provides a user-friendly and powerful way to explore generative AI models in notebooks and improve your productivity in JupyterLab and the Jupyter Notebook. Note that this may not work with VSCode.

## [Tensorflow.js](https://www.tensorflow.org/)
An open-source JavaScript library developed by Google for training and deploying machine learning models in the browser. Enables you to build AI-powered applications without requiring server-side processing.

## [OpenAI API](https://platform.openai.com/docs/introduction)
Provides access to powerful language models like GPT-3, allowing you to integrate natural language processing capabilities into your applications. Enables tasks such as text generation, summarization, sentiment analysis, and code completion.

## [Codeium](https://codeium.com/vscode_tutorial) (A freemium alternative to Github Copilot)
An AI-powered code completion tool that suggests code snippets and entire functions based on the context of the code. Assists you in writing code faster and more efficiently by providing intelligent code suggestions. You can add it by searching for it in the VS Code extensions; once enabled on a "free" account, you'll be able to get auto suggestions right in your code, right there in the IDE. This is essentially the same as Github Copilot, though it may lack integration features with Github that are native to Copilot.

# Activity 3: AI Pair Programming
1. Go to VS Code and enter the lab folder
1. Create a new file called `ai-practice.py`
1. Click on the "Extensions" icon in the tool tray on the far left edge of the UI and search for "Codeium"
1. Install Codeium, then select "restart extensions" to enable it
1. Return to our py file and copy paste the following comment:

    ```python
    # function that accepts two numbers as parameters (num1, num2) and returns an array of results from performing all known mathematical operations on them
    ```

    Now hit enter and wait a moment. What happens?

1. Hit tab a few times. Was the output what you expected? Why or why not?
1. Now let's try out another capability.

    Copy and paste the below code into the editor:

    ```python
    firstName = input("What is your name?")

    print(f"Oh hey there, {firstName}!")
    ```

1. Now select both lines of code and wait for a moment. There will appear above that selected code some grey text links including:

    ```bash
    Codeium: Edit(ctrl+I)
    ```

    Click that link and in the text box that pops up, input the following prompt:

    ```bash
    Refactor this code to include commentary which documents the purpose and functionality of this code without altering the code itself
    ```

1. What did it produce? If you like the commentary, you can now click "Accept all". If not, you can select to discard or hit CTRL/CMD+Z to undo the change.
1. Finally, try asking Codeium to translate some JavaScript code to Python for you!


# Case Studies of Successful AI Implementations

## [Wix ADI](https://support.wix.com/en/article/adi-getting-to-know-the-wix-adi-builder) (Artificial Design Intelligence)
Wix's AI-powered website builder creates fully functional websites based on user preferences and content. Trained on data from millions of users, Wix ADI generates unique website designs and layouts tailored to specific industries and needs.

## [Stitch Fix's Personalized Styling](https://newsroom.stitchfix.com/blog/how-were-revolutionizing-personal-styling-with-generative-ai/)
Stitch Fix uses AI algorithms to analyze customer preferences, style profiles, and feedback to provide personalized clothing recommendations. The AI-driven approach has improved customer satisfaction, retention, and overall business growth.

## [Airbnb's AI-Powered Search and Personalization](https://news.airbnb.com/sharing-more-about-the-technology-that-powers-airbnb/)
Airbnb leverages AI to enhance its search functionality, delivering more relevant and personalized results to users. AI algorithms analyze user behavior, preferences, and search history to provide tailored recommendations and improve the booking experience.

# Key Takeaways 
- Foundational coding knowledge remains essential
- Understanding AI's limitations are key for using it responsibly and successfully